In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


In [7]:
def gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)


def gauss(image):
    return cv2.GaussianBlur(image, (5, 5), 0)


def canny(image):
    edges = cv2.Canny(image, 50, 150)
    return edges


In [8]:
cap = cv2.VideoCapture('april21.avi')
cap.set(150, 0)
ret, frame = cap.read()
gray = gray(frame)


In [9]:
canny = canny(gray)


In [10]:
lines = cv2.HoughLines(canny, rho=1, theta=np.pi/180,
                        threshold=100)



In [11]:

cv2.imshow('hey', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
